In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from glob import glob

In [6]:
files= [file for file in glob('*.csv')]

In [7]:
files

['country_code.csv', 'Summer Olympic medallists 1896 to 2008.csv']

In [12]:
olympic_data= pd.read_csv(files[1],sep=',',skiprows=4, encoding = "ISO-8859-1")

In [13]:
olympic_data.head()

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver


In [14]:
olympic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29216 entries, 0 to 29215
Data columns (total 10 columns):
City            29216 non-null object
Edition         29216 non-null int64
Sport           29216 non-null object
Discipline      29216 non-null object
Athlete         29216 non-null object
NOC             29216 non-null object
Gender          29216 non-null object
Event           29216 non-null object
Event_gender    29216 non-null object
Medal           29216 non-null object
dtypes: int64(1), object(9)
memory usage: 2.2+ MB


In [19]:
olympic_data.shape

(29216, 10)

In [16]:
country= pd.read_csv(files[0])

In [17]:
country.head()

,Country,NOC
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,ALG
3,American Samoa*,ASA
4,Andorra,AND


In [20]:
country.shape

(201, 2)

In [25]:
olympic= pd.merge(olympic_data,country, on='NOC', how='left')

In [26]:
olympic.shape

(29216, 11)

In [27]:
olympic.head()

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal,Country
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold,Hungary
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver,Austria
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze,Greece
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold,Greece
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver,Greece


In [62]:
olympic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29216 entries, 0 to 29215
Data columns (total 11 columns):
City            29216 non-null object
Edition         29216 non-null int64
Sport           29216 non-null object
Discipline      29216 non-null object
Athlete         29216 non-null object
NOC             29216 non-null object
Gender          29216 non-null object
Event           29216 non-null object
Event_gender    29216 non-null object
Medal           29216 non-null object
Country         23857 non-null object
dtypes: int64(1), object(10)
memory usage: 2.7+ MB


In [121]:
#why we have Event_gender and Gender column.. is there any difference?
ev_gen= olympic[['Event_gender','Gender']]

In [122]:
ev_gen.head()

,Event_gender,Gender
0,M,Men
1,M,Men
2,M,Men
3,M,Men
4,M,Men


In [123]:
#find drop duplicate to check how many combination are there
ev_gen= ev_gen.drop_duplicates()

In [124]:
ev_gen

,Event_gender,Gender
0,M,Men
348,X,Men
416,W,Women
639,X,Women
23675,W,Men


In [125]:
# row 23675 has unlikely combination W and Men. Is it a valid record?
olympic.iloc[23675]

City                       Sydney
Edition                      2000
Sport                   Athletics
Discipline              Athletics
Athlete         CHEPCHUMBA, Joyce
NOC                           KEN
Gender                        Men
Event                    marathon
Event_gender                    W
Medal                      Bronze
Country                     Kenya
Name: 23675, dtype: object

In [134]:
#Search on google and found that CHEPCHUMBA, Joyce is women.
olympic.loc[23675,'Gender']='Women'

In [135]:
olympic.iloc[23675]['Gender']

'Women'

In [137]:
olympic_edition= olympic.pivot_table(index=['Edition','City','Country'], columns='Medal', values='Athlete', aggfunc='count')

In [138]:
olympic_edition.head()

Medal                     Bronze  Gold  Silver
Edition City   Country                        
1896    Athens Australia     NaN   2.0     NaN
               Austria       2.0   2.0     1.0
               Denmark       3.0   1.0     2.0
               France        2.0   5.0     4.0
               Germany       2.0  26.0     5.0

In [139]:
olympic_edition=olympic_edition.sort_index(level=['Country'], ascending=True)

In [140]:
olympic_edition.head()

,,Medal,Bronze,Gold,Silver
Edition,City,Country,,,
2008,Beijing,Afghanistan,1.0,NaN,NaN
1984,Los Angeles,Algeria,2.0,NaN,NaN
1992,Barcelona,Algeria,1.0,1.0,NaN
1996,Atlanta,Algeria,1.0,2.0,NaN
2000,Sydney,Algeria,3.0,1.0,1.0


In [141]:
olympic_edition['Grand Total']= olympic_edition.sum(axis=1)

In [142]:
olympic_edition.head()

,,Medal,Bronze,Gold,Silver,Grand Total
Edition,City,Country,,,,
2008,Beijing,Afghanistan,1.0,NaN,NaN,1.0
1984,Los Angeles,Algeria,2.0,NaN,NaN,2.0
1992,Barcelona,Algeria,1.0,1.0,NaN,2.0
1996,Atlanta,Algeria,1.0,2.0,NaN,3.0
2000,Sydney,Algeria,3.0,1.0,1.0,5.0


In [143]:
olympic_edition.loc[(2008,slice(None),['China']),:]

,,Medal,Bronze,Gold,Silver,Grand Total
Edition,City,Country,,,,
2008,Beijing,China,57.0,74.0,53.0,184.0


In [144]:
olympic_edition.shape

(988, 4)

In [156]:
olympic_edition=olympic_edition.reset_index()

In [157]:
olympic_edition.head()

Medal,Edition,City,Country,Bronze,Gold,Silver,Grand Total
0,2008,Beijing,Afghanistan,1.0,NaN,NaN,1.0
1,1984,Los Angeles,Algeria,2.0,NaN,NaN,2.0
2,1992,Barcelona,Algeria,1.0,1.0,NaN,2.0
3,1996,Atlanta,Algeria,1.0,2.0,NaN,3.0
4,2000,Sydney,Algeria,3.0,1.0,1.0,5.0


In [180]:
olympic_by_edition= olympic_edition.groupby('Edition')

In [185]:
olympic_winner_edition=olympic_by_edition['Grand Total'].max()

In [186]:
type(olympic_winner_edition)

pandas.core.series.Series

In [190]:
olympic_winner_edition.head()

Edition
1896     52.0
1900    185.0
1904    394.0
1908    347.0
1912    173.0
Name: Grand Total, dtype: float64

In [196]:
pd.merge(olympic_edition,olympic_winner_edition, on=['Edition','Grand Total'])

,Edition,City,Country,Bronze,Gold,Silver,Grand Total
0,1980,Moscow,Bulgaria,40.0,8.0,46.0,94.0
1,1900,Paris,France,53.0,46.0,86.0,185.0
2,1936,Berlin,Germany,59.0,88.0,63.0,210.0
3,1896,Athens,Greece,22.0,10.0,20.0,52.0
4,1912,Stockholm,Sweden,25.0,86.0,62.0,173.0
5,1908,London,United Kingdom,68.0,144.0,135.0,347.0
6,1904,St Louis,United States,111.0,146.0,137.0,394.0
7,1920,Antwerp,United States,38.0,110.0,45.0,193.0
8,1924,Paris,United States,51.0,110.0,37.0,198.0
9,1928,Amsterdam,United States,17.0,43.0,24.0,84.0
